# **Shopee Code League - Order Brushing**

This notebook is for Shopee Code League Competition, Order Brushing, to detecting abnormal user behaviour. The competition details can be obtained from [here](https://www.kaggle.com/c/order-brushing-shopee-code-league/overview).

Team name: CY Team (Malaysia)

Team member:
1. Chong Li Chuin
2. Ee Yeo Keat

# **Import libraries and read dataset**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from datetime import datetime
from datetime import timedelta
from google.colab import files

In [2]:
from tensorflow.python.client import device_lib
check_device = device_lib.list_local_devices()
check_device

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3915240370635349002, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5559909556502834421
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8692903394793055871
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15701401920
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1578337973528707150
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [3]:
# order_brush_order.csv
d = pd.read_csv('order_brush_order.csv', index_col=0).sort_values(by=['event_time','shopid','userid']).reset_index()
len(d.shopid.unique())

18770

In [4]:
# order_brush_order.csv
data = pd.read_csv('test_brush.csv', index_col=0).sort_values(by=['event_time','shopid','userid']).reset_index()
data

,orderid,shopid,userid,event_time
0,31388857391599,91242967,112494421,2019-12-30 15:07:37
1,31388858606904,85669731,112494421,2019-12-30 15:07:38
2,31388860144322,85669731,112494421,2019-12-30 15:07:41
3,31388862051153,85669731,112494421,2019-12-30 15:07:42
4,31388864143841,68986766,65129646,2019-12-30 15:07:45
5,31388864143840,85669731,10209247,2019-12-30 15:07:45
6,31388864143842,91242967,10209247,2019-12-30 15:07:45
7,31388868340962,68986766,65129646,2019-12-30 15:07:48
8,31388869521738,68986766,65129646,2019-12-30 15:07:49
9,31388867881658,68986766,86829555,2019-12-31 18:07:49


In [5]:
date = data['event_time']
date_time = []
for i in date:
  date_time.append(datetime.strptime(i,"%Y-%m-%d %H:%M:%S"))

In [6]:
data['date_time'] = date_time
data

,orderid,shopid,userid,event_time,date_time
0,31388857391599,91242967,112494421,2019-12-30 15:07:37,2019-12-30 15:07:37
1,31388858606904,85669731,112494421,2019-12-30 15:07:38,2019-12-30 15:07:38
2,31388860144322,85669731,112494421,2019-12-30 15:07:41,2019-12-30 15:07:41
3,31388862051153,85669731,112494421,2019-12-30 15:07:42,2019-12-30 15:07:42
4,31388864143841,68986766,65129646,2019-12-30 15:07:45,2019-12-30 15:07:45
5,31388864143840,85669731,10209247,2019-12-30 15:07:45,2019-12-30 15:07:45
6,31388864143842,91242967,10209247,2019-12-30 15:07:45,2019-12-30 15:07:45
7,31388868340962,68986766,65129646,2019-12-30 15:07:48,2019-12-30 15:07:48
8,31388869521738,68986766,65129646,2019-12-30 15:07:49,2019-12-30 15:07:49
9,31388867881658,68986766,86829555,2019-12-31 18:07:49,2019-12-31 18:07:49


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   orderid     11 non-null     int64         
 1   shopid      11 non-null     int64         
 2   userid      11 non-null     int64         
 3   event_time  11 non-null     object        
 4   date_time   11 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 568.0+ bytes


In [8]:
b = data['date_time'].loc[10]
a = data['date_time'].loc[1]
delt = b-a
delt.total_seconds()

99791.0

In [9]:
data.loc[0:5]

,orderid,shopid,userid,event_time,date_time
0,31388857391599,91242967,112494421,2019-12-30 15:07:37,2019-12-30 15:07:37
1,31388858606904,85669731,112494421,2019-12-30 15:07:38,2019-12-30 15:07:38
2,31388860144322,85669731,112494421,2019-12-30 15:07:41,2019-12-30 15:07:41
3,31388862051153,85669731,112494421,2019-12-30 15:07:42,2019-12-30 15:07:42
4,31388864143841,68986766,65129646,2019-12-30 15:07:45,2019-12-30 15:07:45
5,31388864143840,85669731,10209247,2019-12-30 15:07:45,2019-12-30 15:07:45


In [10]:
check = data.loc[0:5].groupby('shopid')
check.first()

,orderid,userid,event_time,date_time
shopid,,,,
68986766,31388864143841,65129646,2019-12-30 15:07:45,2019-12-30 15:07:45
85669731,31388858606904,112494421,2019-12-30 15:07:38,2019-12-30 15:07:38
91242967,31388857391599,112494421,2019-12-30 15:07:37,2019-12-30 15:07:37


In [11]:
what_user = []
lol = pd.DataFrame(data.loc[0:5].groupby('userid')['userid'].count().reset_index(name='user_count'))
lol['user_count'].apply(lambda ui : print(lol['user_count']))

0    1
1    1
2    4
Name: user_count, dtype: int64
0    1
1    1
2    4
Name: user_count, dtype: int64
0    1
1    1
2    4
Name: user_count, dtype: int64


0    None
1    None
2    None
Name: user_count, dtype: object

In [12]:
lol

,userid,user_count
0,10209247,1
1,65129646,1
2,112494421,4


In [13]:
lol['user_count'].idxmax()

2

In [14]:
key = list(check.groups.keys())
key

[68986766, 85669731, 91242967]

In [15]:
key[1]

85669731

In [16]:
len(key)

3

In [17]:
for k in key:
  print(k)

68986766
85669731
91242967


In [18]:
check_order = check.get_group(key[1])
order = len(check_order.orderid)
buyers = len(check_order.userid.unique())
concentrate_rate = order/buyers
print(concentrate_rate)

2.0


In [19]:
charc = 'x'
haha = 'abc'
a_dict = {}
if charc in a_dict:
  a_dict[charc].append(haha)
else:
  a_dict[charc] = [haha]
print(a_dict)

{'x': ['abc']}


In [ ]:
ans_dic= {}

for i in data.index:
  initial = data['date_time'].loc[i]
  for j in range(i, data.last_valid_index()+1):
    delta = data['date_time'].loc[j] - initial
    if(delta.total_seconds()<3600):
      check = data.loc[i:j].groupby('shopid')
      check['userid'].count()
      user_num = pd.DataFrame(data.loc[0:5].groupby('userid')['userid'].count().reset_index(name='user_count'))
      max_index = user_num['user_count'].idxmax()
      key = list(check.groups.keys())
      for k in key:
        check_order = check.get_group(k)
        order = len(check_order.orderid)
        buyers = len(check_order.userid.unique())
        concentrate_rate = order/buyers
        if(concentrate_rate >= 3):
          if max_index >= 3:


SyntaxError: ignored

In [ ]:
for i in test.index:
  initial = test['date_time'].loc[i]
  for j in range(i, test.index):
    delta = test['date_time'].loc[j] - initial
    print(delta)

In [ ]:
def check_concentrate_rate():
  concentrate_rate = #Number of Orders within 1 hour / Number of Unique Buyers within 1 hour
  if concentrate_rate >= 3:
    shop_list.append(shopid)
  
  return shop_list

In [ ]:
df.to_csv('CY_Team_submission4.csv') 
files.download('CY_Team_submission4.csv')

In [ ]:
num

NameError: ignored

In [ ]:
import pandas as pd
import numpy as np


def add_two(num, a, b):

  x = num + 2 + a + b
  y = a + b

  return x, b

(14, 6)

In [ ]:
y = add_two(6, 2, 5)

In [ ]:
y

(15, 5)

In [ ]:
y

5

In [ ]:
a + 2
b + 2
c + 2


In [ ]:
def say_hi():
  print("Hi, how are you?")


say_hi()

Hi, how are you?


In [ ]:
def RemainingMinSet(remain_kmer, remain_Seq):
  a = 0

  while(len(remain_kmer) != 0):
      
      A = make_automaton(remain_kmer)
      
      matching_file = 'match/matching'+str(a)
      remain_kmer_file = 'match/remain_kmer'+str(a)
      
      # save matching to file
      with open(matching_file, 'w') as f:
          for index in range(len(remain_Seq)):
              x = remain_Seq[index].id
              y = find_matching(remain_Seq[index].seq, A)
              z = len(y)
              f.write(x + ';' + str(y) + ';' + str(z) + '\n')
              if z == 0:
                  for i in range(len(remain_Seq_copy)):
                      if remain_Seq_copy[i].id == x:
                          del remain_Seq_copy[i]
                          break                
      
      remain_Seq = remain_Seq_copy.copy()
      
      df = pd.read_csv(matching_file, delimiter=';', names=['sequence_id', 'matched_kmer', 'count']).sort_values(by='count',ascending=False, kind='mergesort')
      df['matched_kmer'] = df['matched_kmer'].str.replace(r"\[|\]|'","")
      
      fileZ = open('fileZ.txt', 'a') # file Z is an example for output (exampleoutput.txt)
      fileZ.write(df['sequence_id'].iloc[0] + '\n')
      
      kmer_to_remove = df['matched_kmer'].iloc[0].split(', ')
      remain_kmer = list(set(remain_kmer) - set(kmer_to_remove))
      
      with open(remain_kmer_file, 'w') as f:
        for i in remain_kmer:
          f.write(i + '\n')
          
      a = a + 1

In [ ]:
from Bio import SeqIO
from concurrent.futures import ProcessPoolExecutor
import math

remain_Seq = list(SeqIO.parse("remainingSeq.fasta","fasta"))
remain_kmer = [line.rstrip('\n') for line in open ("remainingKmer.txt")]

In [ ]:
RemainingMinSet(remain_kmer, remain_Seq)

In [ ]:
for i in range (1,11):
  print(i)

1
2
3
4
5
6
7
8
9
10
